In [1]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
import math
import copy

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
###############################################################################
# HELPER FUNCTIONS
###############################################################################

def homogeneous(ROLL, PITCH, YAW, X, Y, Z):
    T = np.eye(4,4);
    T[0,0] = math.cos(YAW)*math.cos(PITCH);
    T[0,1] = math.cos(YAW)*math.sin(PITCH)*math.sin(ROLL)-math.sin(YAW)*math.cos(ROLL);
    T[0,2] = math.cos(YAW)*math.sin(PITCH)*math.cos(ROLL)+math.sin(YAW)*math.sin(ROLL);
    T[0,3] = X;
    T[1,0] = math.sin(YAW)*math.cos(PITCH);
    T[1,1] = math.sin(YAW)*math.sin(PITCH)*math.sin(ROLL)+math.cos(YAW)*math.cos(ROLL);
    T[1,2] = math.sin(YAW)*math.sin(PITCH)*math.cos(ROLL)-math.cos(YAW)*math.sin(ROLL);
    T[1,3] = Y;
    T[2,0] = -math.sin(PITCH);
    T[2,1] = math.cos(PITCH)*math.sin(ROLL);
    T[2,2] = math.cos(PITCH)*math.cos(ROLL);
    T[2,3] = Z;
    T[3,3] = 1;
    return T

In [3]:
# .stl file path
fileName = "../../PLuM/sample_data/FLATFOOT_StanfordBunny_jmil_HIGH_RES_Smoothed.stl"

# point cloud sensor
phi = np.linspace(-90,90,20) # one degree increments
theta = np.linspace(0,90,20)  # one degree increments
radius = 1
origin = [0,0,0]

sensorToToModel = homogeneous(0,0,30*math.pi/180,30,20,-5)

In [4]:
# Load mesh and convert to open3d.t.geometry.TriangleMesh
mesh = o3d.io.read_triangle_mesh(fileName)
mesh = o3d.t.geometry.TriangleMesh.from_legacy(mesh)

# Create a scene and add the triangle mesh
scene = o3d.t.geometry.RaycastingScene()
_ = scene.add_triangles(mesh)  # we do not need the geometry ID for mesh

# min_bound = mesh.vertex.positions.min(0).numpy()
# max_bound = mesh.vertex.positions.max(0).numpy()
# print(min_bound)
# print(max_bound)

## transform the mesh to the correct frame
mesh_t = copy.deepcopy(mesh).transform(sensorToToModel)

## Create a scene and add the triangle mesh
scene = o3d.t.geometry.RaycastingScene()
_ = scene.add_triangles(mesh_t)  # we do not need the geometry ID for mesh

In [5]:
# Raycasting

# create the rays
raysStartEnd = []
anglesInOrder = []
for heading in phi:
    for elevation in theta:
        endX = radius*math.sin(elevation*math.pi/180)*math.cos(heading*math.pi/180)
        endY = radius*math.sin(elevation*math.pi/180)*math.sin(heading*math.pi/180)
        endZ = radius*math.cos(elevation*math.pi/180)
        ray = [origin[0],origin[1],origin[2],endX,endY,endZ]
        anglesInOrder.append([heading,elevation])
        raysStartEnd.append(ray)
rays = o3d.core.Tensor(raysStartEnd,
                       dtype=o3d.core.Dtype.Float32)

ans = scene.cast_rays(rays)
rayLengths = ans['t_hit'].numpy()

# convert to Cartesian intersection points
raycastIntersectionPts  = []

counter = 0
for ray in raysStartEnd:
    if not math.isinf(rayLengths[counter]):
        radius = rayLengths[counter]
        heading = anglesInOrder[counter][0]
        elevation = anglesInOrder[counter][1]
        
        intersectX = radius*math.sin(elevation*math.pi/180)*math.cos(heading*math.pi/180)+origin[0];
        intersectY = radius*math.sin(elevation*math.pi/180)*math.sin(heading*math.pi/180)+origin[1];
        intersectZ = radius*math.cos(elevation*math.pi/180)+origin[2]; 
        
        raycastIntersectionPts.append([intersectX,intersectY,intersectZ])
    counter = counter + 1
     
# print(len(rays))
# print(len(raycastIntersectionPts))
pts = np.array(raycastIntersectionPts)
np.savetxt('bunnyRaycast1', pts, delimiter=' ')

